In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import gravity_model as gm

In [2]:
# Configurations
data_dir = "data_dir"
country = "US"
cities = ["NYC", "LAX", "CHI", "DAL", "HOU", "WAS", "MIA", "PHL", "ATL", "PHX", "BOS", "SFR"]

### Figure 1

In [3]:
city = "CHI"

model = gm.gravity_model(data_dir, country, city, cost="rij")
model.exponent_matrix(rmin=1, rmax=50, mbins=10, rbins=100, bin_mode="equal_mass")

mob_data = pd.merge(model.mob_data, model.pop_data.rename(columns={"id": "o", "bin": "bin_o"})[["o", "bin_o"]], on="o", how="left")
mob_data = pd.merge(mob_data, model.pop_data.rename(columns={"id": "d", "bin": "bin_d"})[["d", "bin_d"]], on="d", how="left")

plt.rcParams['font.family'] = 'Arial'
plt.rcParams["figure.figsize"] = (7, 7)

cmap = plt.cm.Set3

Calculating exponent: 100it [01:14,  1.35it/s]


### Figure 1(d)
Distance exponent estimation for the whole dataset.

In [ ]:
data = mob_data.copy()
    
x_bin = [
    (x1 + x2) / 2
    for x1, x2 in zip(model.bin_distance[:-1], model.bin_distance[1:])
]

y_bin_list = [
    [
        y_val
        for x_val, y_val in zip(data.rij, data.rescaled_tij)
        if (x_val >= model.bin_distance[i])
        and (x_val < model.bin_distance[i + 1])
    ]
    for i in range(100)
]
x_bin = [x_val for x_val, y_vals in zip(x_bin, y_bin_list) if len(y_vals) != 0]
y_bin = [np.mean(y_vals) for y_vals in y_bin_list if len(y_vals) != 0]

X_bin = np.array([np.log(x) for x in x_bin]).reshape(-1, 1)
Y_bin = np.array([np.log(y) for y in y_bin]).reshape(-1, 1)

X = np.log(data.rij)
Y = np.log(data.rescaled_tij)

X_ = sm.add_constant(X)

result = sm.OLS(Y, X_).fit()

a = result.params[1]
b = result.params[0]

sns.kdeplot(X, Y, alpha=0.7, cmap="Greys", shade=True, thresh=0.3)
plt.plot([min(X), max(X)], [a*min(X)+b, a*max(X)+b], c="k", ls="--", alpha=0.5, label=label, linewidth=2)

plt.xlabel("Distance $r_{ij} (km)$", fontsize=28)
plt.ylabel("Rescaled traffic $T_{ij}/T_iT_j$", fontsize=28)
plt.xticks([np.log(2), np.log(3), np.log(4), np.log(5), np.log(6), np.log(7), np.log(8), np.log(9), np.log(10), np.log(20), np.log(30), np.log(40)], ["2", "", "4", "", "6", "", "8", "", "10", "20", "", "40"], fontsize=24)
plt.yticks([np.log(0.01), np.log(0.001), np.log(0.002), np.log(0.003), np.log(0.004), np.log(0.005), np.log(0.006), np.log(0.007), np.log(0.008), np.log(0.009), np.log(0.0001), np.log(0.0002), np.log(0.0003), np.log(0.0004), np.log(0.0005), np.log(0.0006), np.log(0.0007), np.log(0.0008), np.log(0.0009), np.log(0.00009), np.log(0.00008), np.log(0.00007), np.log(0.00006), np.log(0.00005)], ["$10^{-2}$", "$10^{-3}$", "", "", "", "", "", "", "", "", "$10^{-4}$", "", "", "", "", "", "", "", "", "", "", "", "", ""], fontsize=24)
plt.ylim([-10, -5.5])

plt.show()

/home/ohhyun/.pyenv/versions/3.6.8/envs/ohhyun/lib/python3.6/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


### Figure 1(e)
Distance exponents estimations for different population groups.

In [ ]:
cmaps_bin = ["Reds", "Oranges", "Greens", "Blues", "Purples"]
cmaps_plot = ["darkred", "sienna", "darkolivegreen", "darkblue", "purple"]

for n, i in enumerate(range(0, 10, 2)):
    data = mob_data[(mob_data.bin_o == i) & (mob_data.bin_d == i)]
    
    x_bin = [
        (x1 + x2) / 2
        for x1, x2 in zip(model.bin_distance[:-1], model.bin_distance[1:])
    ]

    y_bin_list = [
        [
            y_val
            for x_val, y_val in zip(data.rij, data.rescaled_tij)
            if (x_val >= model.bin_distance[i])
            and (x_val < model.bin_distance[i + 1])
        ]
        for i in range(100)
    ]
    x_bin = [x_val for x_val, y_vals in zip(x_bin, y_bin_list) if len(y_vals) != 0]
    y_bin = [np.mean(y_vals) for y_vals in y_bin_list if len(y_vals) != 0]

    X_bin = np.array([np.log(x) for x in x_bin]).reshape(-1, 1)
    Y_bin = np.array([np.log(y) for y in y_bin]).reshape(-1, 1)
    
    X = np.log(data.rij)
    Y = np.log(data.rescaled_tij)
    
    df = pd.DataFrame([X, Y]).T
    df["bin"] = i
    
    X_ = sm.add_constant(X)
    
    if len(X_) > 0:
        result = sm.OLS(Y, X_).fit()

        a = result.params[1]
        b = result.params[0]
    else:
        a = 0
        b = 0
    
    label = "Group {0}".format(i+1)
    sns.kdeplot(X, Y, alpha=0.4, cmap=cmaps_bin[n], shade=True, thresh=0.3)
    if a != 0:
        plt.plot([min(X), max(X)], [a*min(X)+b, a*max(X)+b], c=cmaps_plot[n], ls="--", alpha=0.5, label=label, linewidth=2)

plt.xlabel("Distance $r_{ij}$ (km)", fontsize=28)
plt.ylabel("Rescaled traffic $T_{ij}/T_iT_j$", fontsize=28)
plt.legend(fontsize=18)
plt.legend(["Group {}".format(i+1) for i in range(0, 10, 2)], fontsize=14)
plt.xticks([np.log(2), np.log(3), np.log(4), np.log(5), np.log(6), np.log(7), np.log(8), np.log(9), np.log(10), np.log(20), np.log(30), np.log(40)], ["2", "", "4", "", "6", "", "8", "", "10", "20", "", "40"], fontsize=24)
plt.yticks([np.log(0.01), np.log(0.001), np.log(0.002), np.log(0.003), np.log(0.004), np.log(0.005), np.log(0.006), np.log(0.007), np.log(0.008), np.log(0.009), np.log(0.0001), np.log(0.0002), np.log(0.0003), np.log(0.0004), np.log(0.0005), np.log(0.0006), np.log(0.0007), np.log(0.0008), np.log(0.0009), np.log(0.00009), np.log(0.00008), np.log(0.00007), np.log(0.00006), np.log(0.00005)], ["$10^{-2}$", "$10^{-3}$", "", "", "", "", "", "", "", "", "$10^{-4}$", "", "", "", "", "", "", "", "", "", "", "", "", ""], fontsize=24)
plt.ylim([-10, -5.5])

plt.show()

### Figure 2, 5, 6

In [ ]:
plt.rcParams['font.family'] = 'Arial'

data_cols = ["gamma", "rsq", "dist_avg", "dist_var", "dist_fra"]

figures = {col: plt.subplots(nrows=3, ncols=4, figsize=(40, 21)) for col in data_cols}

df_data = pd.DataFrame()

for i, city in enumerate(cities):
    print(city)
    model = gm.gravity_model(data_dir, country, city, cost="rij")
    model.exponent_matrix(rmin=1, rmax=50, mbins=1)
    
    mono_data = {col: model.matrix[col][0][0] for col in data_cols}
    
    model.exponent_matrix(rmin=1, rmax=50, mbins=10)

    for col in data_cols:
        model.plot_matrix(figures[col][0], figures[col][1][int(i/4)][i%4], param=col, ref=mono_data[col])

for f in figures.values():
    f[0].show()